In [1]:
#Importing of necessary packages
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import re
import nltk
import emoji
import demoji

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.cluster import KMeans

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, recall_score, precision_score, make_scorer
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertModel, pipeline
from numpy import loadtxt, savetxt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D, BatchNormalization, Bidirectional
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score

#import bayes opt for hyperparameter tuning
from bayes_opt import BayesianOptimization

#We are going to use word vectorization to embed the words
import gensim
from gensim import downloader
import pickle

2023-04-04 00:00:06.739649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Bag of Words model
We shall start off with the simplest model for test classfication, ie a bag of words model, which simply stores the count of each word in the vocabulary for each tweet, which results in a rather sparse representation of each tweet

In [2]:
df_tweets = pd.read_csv('tweets_dataset_processed.csv')

In [3]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cleaned_text    50000 non-null  object
 1   isBot           50000 non-null  int64 
 2   tokenized       50000 non-null  object
 3   processed_data  50000 non-null  object
 4   sentence        49911 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


In [4]:
df_tweets = df_tweets.dropna()

In [5]:
df_tweets[df_tweets['sentence'].isna()]

,cleaned_text,isBot,tokenized,processed_data,sentence


In [6]:
#Now we create the BOW vectors
#Next, we initialize a count vectorizer and transform the text into Bag of Words vectors, if some word appears in more than half, we ignore that word
vectorizer = CountVectorizer(max_df = 0.5, min_df = 10)
#Create an array that contains the BOW representation of each tweet
bow_array  = vectorizer.fit_transform(df_tweets['sentence'])
#Converting the result into a pandas dataframe
bow = pd.DataFrame(bow_array.toarray(),columns = vectorizer.get_feature_names_out())

In [7]:
#Next, treat the BOW vectors as features and split the data into training and testing sets
X = bow
y = df_tweets['isBot']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [8]:
#Instantiate and train a logistic regression model
model_log_reg_bow = LogisticRegression(max_iter = 100000)

model_log_reg_bow.fit(X_train, y_train)

LogisticRegression(max_iter=100000)

In [9]:
y_pred = model_log_reg_bow.predict(X_test)

# Error Metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
lg_loss = metrics.log_loss(y_test, y_pred)
print(f'Log Loss: {lg_loss}')
roc_auc = metrics.roc_auc_score(y_test, y_pred)
print(f'ROC AUC: {roc_auc}')
f1_score = metrics.f1_score(y_test, y_pred)
print(f'F1-score: {f1_score}')
precision = metrics.precision_score(y_test, y_pred)
print(f'Precision: {precision}')
recall = metrics.recall_score(y_test, y_pred)
print(f'Recall: {recall}')
report = metrics.classification_report(y_test, y_pred)
print(f'Classification Report: \n {report}')

# print(classification_report(y_test, y_pred))

Accuracy: 0.6892717619953922
Log Loss: 11.199780908849183
ROC AUC: 0.6891969611404714
F1-score: 0.6828220858895706
Precision: 0.6941787941787941
Recall: 0.671830985915493
Classification Report: 
               precision    recall  f1-score   support

           0       0.68      0.71      0.70      5013
           1       0.69      0.67      0.68      4970

    accuracy                           0.69      9983
   macro avg       0.69      0.69      0.69      9983
weighted avg       0.69      0.69      0.69      9983



In [10]:
#Let us calculate the score under the ROC curve for the logistic regression model with BOW
probabilities = model_log_reg_bow.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, probabilities))

0.7627302012754764


In [11]:
#Try fitting a more complex function like the random forest for this problem
model_rf_bow = RandomForestClassifier()
model_rf_bow.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
#print the classification report for random forest
y_pred = model_rf_bow.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.70      0.72      5013
           1       0.72      0.75      0.73      4970

    accuracy                           0.73      9983
   macro avg       0.73      0.73      0.73      9983
weighted avg       0.73      0.73      0.73      9983



In [13]:
#Calculating the ROC score for the random forest
probabilities = model_rf_bow.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, probabilities))

0.8162802267424616
